In [1]:
import os
import pandas as pd
import numpy as np
import scipy.optimize as optimize
#data reading and cleaning
import gc
from tqdm import tqdm
print('Preprocessing may take about 5 minutes')
items = pd.read_csv('Data/items.csv')
samples = pd.read_csv('Data/samples.csv')
test = pd.read_csv('Data/test.csv')
train = pd.read_csv('Data/train.csv')
users = pd.read_csv('Data/users.csv')


train_cols ={
    'User ID':'usr_id',
    'Item1 ID':'item1_id',
    'Item2 ID':'item2_id',
    'Preference':'preference'
}
train = train.rename(columns=train_cols)

test_cols ={
    'User ID':'usr_id',
    'Item1 ID':'item1_id',
    'Item2 ID':'item2_id',
}
test = test.rename(columns=test_cols)

users_cols ={
    'User ID':'usr_id',
    ' Education':'education',
    ' Age':'age',
    ' Gender':'gender',
    ' Region':'region'
}
users = users.rename(columns=users_cols)

# data merging

item1_cols ={
    'Item ID':'item1_id',
    ' BodyType':'body_type_1',
    ' Transmission':'transmission_1',
    ' Engin Capacity':'engin_capacity_1',
    ' Fuel Consumed':'fuel_consumed_1'
}

item2_cols ={
    'Item ID':'item2_id',
    ' BodyType':'body_type_2',
    ' Transmission':'transmission_2',
    ' Engin Capacity':'engin_capacity_2',
    ' Fuel Consumed':'fuel_consumed_2'
}

def find_winner():
    k = pd.read_csv('Data/train.csv')
    train_cols ={
        'User ID':'usr_id',
        'Item1 ID':'item1_id',
        'Item2 ID':'item2_id',
        'Preference':'preference'
    }
    k = k.rename(columns=train_cols)

    win=list()
    for index in k.index:
        if k.preference[index] ==0:
            win.append(k.item1_id[index])
        else:
            win.append(k.item2_id[index])
    k['win'] = win
    
    return k

def number_count(k):
    k = k.drop(columns=['preference','win'])

    p1 = k.pivot_table(index = ['usr_id'],columns=['item1_id'], aggfunc='count').fillna(0)
    p2= k.pivot_table(index = ['usr_id'],columns=['item2_id'], aggfunc='count').fillna(0)

    p1.columns = p1.columns.to_series().str.join('_')
    p2.columns = p2.columns.to_series().str.join('_')

    p1.columns = ['1_count','2_count','3_count','4_count','5_count',
                 '6_count','7_count','8_count','9_count','10_count']

    p2.columns = ['1_count','2_count','3_count','4_count','5_count',
                 '6_count','7_count','8_count','9_count','10_count']
    del p1.index.name, p2.index.name
    return p1+p2

def winner_count(k):
    k = k.drop(columns=['item1_id','item2_id'])

    result = k.pivot_table(index = ['usr_id'],columns=['win'], aggfunc='count')

    result.columns.to_series().str.join('_')
    del result.index.name

    #result.columns = result.columns.get_level_values(0)
    result.columns = ['1_count','2_count','3_count','4_count','5_count',
                 '6_count','7_count','8_count','9_count','10_count']
    return result

def occur_rate(k):
    k = (k/9).fillna(0)
    k['usr_id'] = k.index
    
    return k

def win_rate(w_count,n_count):
    k = (w_count/n_count).fillna(0)
    k.columns = ['1_win','2_win','3_win','4_win','5_win',
                '6_win','7_win','8_win','9_win','10_win']
    k['usr_id'] = k.index
    
    return k
    
def max_f(params):
    r1,r2,r3,r4,r5,r6,r7,r8,r9,r10 = params
    R = [r1,r2,r3,r4,r5,r6,r7,r8,r9,r10]
    index:int
    result = 1
    for index in data.index:
        result *= R[data.win[index]-1] / (R[data.item1_id[index]-1] + R[data.item2_id[index]-1])
        
    return -result


#calculate occur times and win rate

winner_list = find_winner()
n_count = number_count(winner_list)
w_count = winner_count(winner_list)
occur = occur_rate(n_count)
win = win_rate(w_count,n_count)

#concat train and test
df = pd.concat([train,test])
item_1 = items.copy().rename(columns = item1_cols)
item_2 = items.copy().rename(columns = item2_cols)
df = pd.merge(df,users, on =['usr_id'])
df = pd.merge(df,item_1, on =['item1_id'])
df = pd.merge(df,item_2, on =['item2_id'])
df.shape

del item_1,item_2
gc.collect()
#feature generation
#df = pd.merge(df,occur,on = ['usr_id'])
df = pd.merge(df,win,on = ['usr_id'])
df['engin2_bigger'] = 0
df.engin2_bigger[df.engin_capacity_1==df.engin_capacity_2] = 2
df.engin2_bigger[df.engin_capacity_1>df.engin_capacity_2] = 0
df.engin2_bigger[df.engin_capacity_1<df.engin_capacity_2] = 1

del n_count,w_count,occur,win
gc.collect()

#calculate likelyhood

r_cobyla_list = list()
r_powell_list = list()
initial_guess = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
for i in tqdm(range(60)):
    data = winner_list[winner_list['usr_id']==i]
    result_cobyla = optimize.minimize(max_f, initial_guess, method='COBYLA')
    result_powell = optimize.minimize(max_f, initial_guess, method='Powell')
    r_cobyla_list.append(result_cobyla.x - result_cobyla.x.min() + 1)
    r_powell_list.append(result_powell.x - result_powell.x.min()+1)
    

powell = list()
cobylab = list()
for index in tqdm(df.index):
    R_powell = r_powell_list[df.usr_id[index]-1]
    R_cobylab = r_cobyla_list[df.usr_id[index]-1]
    prob_p = R_powell[df.item2_id[index]-1] / (R_powell[df.item1_id[index]-1] + R_powell[df.item2_id[index]-1])
    prob_c = R_powell[df.item2_id[index]-1] / (R_powell[df.item1_id[index]-1] + R_powell[df.item2_id[index]-1])
    
    powell.append(prob_p)
    cobylab.append(prob_c)
    

df['powell'] = powell
#df['cobylab'] = cobylab

del powell#, cobylab
gc.collect()
#split train test
test = df[df.preference.isna()]
train = df[df.preference.isna()==0]

label = train.preference

train = train.drop(columns=['preference'])
test = test.drop(columns=['preference'])

#train = train.drop(columns=['preference','usr_id','item1_id','item2_id'])
#test = test.drop(columns=['preference','usr_id','item1_id','item2_id'])
#train_test split

from sklearn.model_selection import train_test_split
train_data, valid_data, train_target, valid_target = train_test_split(train, label, test_size=0.33, random_state=9)

Preprocessing may take about 5 minutes


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:157: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 13%|█▎        | 8/60 [00:45<04:52,  5.63s/it]/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:128: RuntimeWarning: divide by zero encountered in do

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
classifier = LogisticRegression(C= 5.671226331427315, penalty= 'l1')  # 使用类，参数全是默认的  
classifier.fit(train_data, train_target)  # 训练数据来学习，不需要返回值  
#print(accuracy_score(valid_target, classifier.predict(valid_data)))

lv2_logit = classifier.predict_proba(valid_data)[:,1]
lv2_logit_test = classifier.predict_proba(test)[:,1]
score = accuracy_score(valid_target,classifier.predict(valid_data))
print('Logistic Regression Accuracy:{}'.format(score))

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(leaf_size= 29, n_neighbors= 9, p= 1, weights= 'distance')
neigh.fit(train_data,train_target) 

lv2_knn = neigh.predict_proba(valid_data)[:,1]
lv2_knn_test = neigh.predict_proba(test)[:,1]
score = accuracy_score(valid_target, neigh.predict(valid_data))
print('KNN Accuracy:{}'.format(score))

import lightgbm as lgb

#test = test.drop(columns=['User-Item1-Item2','Preference'])
clf = lgb.LGBMClassifier(bagging_fraction= 0.6508296012331772, bagging_freq= 1, boost= 'gbdt',
                         feature_fraction= 0.3474516582970086, learning_rate= 0.16475833466929507,
                         metric= 'binary_logloss', min_data_in_leaf= 52, num_leaves= 40, num_threads= 2,
                         objective= 'binary', tree_learner= 'data')

clf.fit(train_data, train_target)

lv2_lgb = clf.predict_proba(valid_data)[:,1]
lv2_lgb_test = clf.predict_proba(test)[:,1]
score = accuracy_score(valid_target, clf.predict(valid_data))
print('LGBM Accuracy:{}'.format(score))

from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
NB = gnb.fit(train_data, train_target)

lv2_nb = NB.predict_proba(valid_data)[:,1]
lv2_nb_test = NB.predict_proba(test)[:,1]
score = accuracy_score(valid_target, NB.predict(valid_data))
print('Naive Bayes Accuracy:{}'.format(score))

from sklearn.svm import SVC
svc = SVC(C= 4.460710230756742, gamma= 0.013244596886327797, kernel= 'poly',probability=True)
svm = svc.fit(train_data, train_target)

lv2_svm = svm.predict_proba(valid_data)[:,1]
lv2_svm_test = svm.predict_proba(test)[:,1]
score = accuracy_score(valid_target, svm.predict(valid_data))
print('SVM Accuracy:{}'.format(score))

'''# Create your first MLP in Keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

# create model

model = Sequential()
model.add(Dense(128, input_dim=27, activation='relu'))
model.add(Dense(36, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(8, activation='relu'))
#model.add(Dropout(p=0.01))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
model.fit(train_data, train_target, epochs=10, batch_size=10)
# evaluate the model
scores = model.evaluate(valid_data, valid_target)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

lv2_keras = model.predict(valid_data)
lv2_keras_test = model.predict(test)'''

from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(activation= 'tanh', alpha= 1.6005926425406648e-08, hidden_layer_sizes= 83, solver= 'adam',
                   random_state=42)
mlp.fit(train_data, train_target)

lv2_nn = mlp.predict_proba(valid_data)[:,1]
lv2_nn_test = mlp.predict_proba(test)[:,1]
score = accuracy_score(valid_target, mlp.predict(valid_data))
print('Neural Network Accuracy:{}'.format(score))

from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(bootstrap= 0, criterion= 'entropy', max_depth=24,
                                    max_features= 0.3090928393734417, min_samples_leaf= 2,
                                    n_estimators= 30, random_state= 40)
classifier.fit(train_data, train_target)  

lv2_RF = classifier.predict_proba(valid_data)[:,1]
lv2_RF_test = classifier.predict_proba(test)[:,1]
score = accuracy_score(valid_target, classifier.predict(valid_data))
print('Random Forest Accuracy:{}'.format(score))

lv2_df = np.column_stack((lv2_logit,lv2_knn,lv2_lgb,lv2_nb,lv2_nn,lv2_svm,lv2_RF))
lv2_df = pd.DataFrame(lv2_df,columns=['logit', 'knn', 'lgb', 'nb','nn','svm','rf'])

qtarget = valid_target.reset_index()['preference']
#lv2_df['preference'] = qtarget
lv2_train_data, lv2_valid_data, lv2_train_target, lv2_valid_target = train_test_split(lv2_df, qtarget, test_size=0.33, random_state=9)

lv2_df_test = np.column_stack((lv2_logit_test,lv2_knn_test,lv2_lgb_test,lv2_nb_test,lv2_nn_test
                               ,lv2_svm_test,lv2_RF_test))
lv2_df_test = pd.DataFrame(lv2_df_test,columns=['logit', 'knn', 'lgb', 'nb','nn','svm','rf'])

Logistic Regression Accuracy:0.6274944567627494
KNN Accuracy:0.7095343680709535


/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


LGBM Accuracy:0.8713968957871396
Naive Bayes Accuracy:0.623059866962306
SVM Accuracy:0.8248337028824834
Neural Network Accuracy:0.8159645232815964
Random Forest Accuracy:0.8226164079822617


In [3]:
from sklearn.metrics import accuracy_score
from hyperopt import fmin, tpe, hp, rand
from sklearn.neural_network import MLPClassifier 

parameter_space_NN = {'hidden_layer_sizes':hp.randint('hidden_layer_sizes', 99)+1,
         'alpha':hp.lognormal('alpha', mu=np.log(1e-4), sigma=1),
         'solver':hp.choice('solver', ['lbfgs', 'sgd', 'adam']),
         'activation':hp.choice('activation', ['logistic', 'tanh', 'relu']),'random_state':42
         #'learning_rate':hp.uniform('learning_rate', low=0.001, high=0.999),
        }

count = 0

def function(args):
    print(args)

    # **可以把dict转换为关键字参数，可以大大简化复杂的函数调用
    clf = MLPClassifier (**args)

    # 训练模型
    clf.fit(lv2_train_data,lv2_train_target)

    # 预测测试集
    prediction = clf.predict(lv2_valid_data)

    global count
    count = count + 1
    score = accuracy_score(lv2_valid_target, prediction)
    print("第%s次，测试集正确率为：" % str(count),score)

    # 由于hyperopt仅提供fmin接口，因此如果要求最大值，则需要取相反数
    return -score

# algo指定搜索算法，目前支持以下算法：
# ①随机搜索(hyperopt.rand.suggest)
# ②模拟退火(hyperopt.anneal.suggest)
# ③TPE算法（hyperopt.tpe.suggest，算法全称为Tree-structured Parzen Estimator Approach）
# max_evals指定枚举次数上限，即使第max_evals次枚举仍未能确定全局最优解，也要结束搜索，返回目前搜索到的最优解
best = fmin(function, parameter_space_NN, algo=tpe.suggest, max_evals=10000)

# best["kernel"]返回的是数组下标，因此需要把它还原回来
solver_list = ['lbfgs', 'sgd', 'adam']
best["solver"] = solver_list[best["solver"]]

activation_list = ['logistic', 'tanh', 'relu']
best["activation"] = activation_list[best["activation"]]

print("最佳参数为：",best)

{'activation': 'logistic', 'alpha': 3.7943122965693006e-05, 'hidden_layer_sizes': 5, 'random_state': 42, 'solver': 'lbfgs'}
第1次，测试集正确率为： 0.8389261744966443
{'activation': 'relu', 'alpha': 3.41635571128461e-05, 'hidden_layer_sizes': 37, 'random_state': 42, 'solver': 'adam'}


/usr/local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


第2次，测试集正确率为： 0.8456375838926175
{'activation': 'logistic', 'alpha': 0.00022633869292574478, 'hidden_layer_sizes': 38, 'random_state': 42, 'solver': 'sgd'}
第3次，测试集正确率为： 0.5302013422818792
{'activation': 'relu', 'alpha': 4.364410427124662e-05, 'hidden_layer_sizes': 55, 'random_state': 42, 'solver': 'lbfgs'}
第4次，测试集正确率为： 0.7919463087248322
{'activation': 'relu', 'alpha': 0.0004256096461830934, 'hidden_layer_sizes': 90, 'random_state': 42, 'solver': 'sgd'}
第5次，测试集正确率为： 0.8187919463087249
{'activation': 'logistic', 'alpha': 0.0001518048780991098, 'hidden_layer_sizes': 49, 'random_state': 42, 'solver': 'adam'}
第6次，测试集正确率为： 0.8456375838926175
{'activation': 'logistic', 'alpha': 9.56919620007268e-05, 'hidden_layer_sizes': 26, 'random_state': 42, 'solver': 'lbfgs'}
第7次，测试集正确率为： 0.8456375838926175
{'activation': 'logistic', 'alpha': 5.599014816850125e-05, 'hidden_layer_sizes': 11, 'random_state': 42, 'solver': 'sgd'}
第8次，测试集正确率为： 0.5302013422818792
{'activation': 'relu', 'alpha': 0.0001620906770

第57次，测试集正确率为： 0.7248322147651006
{'activation': 'tanh', 'alpha': 3.242385245520244e-08, 'hidden_layer_sizes': 66, 'random_state': 42, 'solver': 'sgd'}
第58次，测试集正确率为： 0.8523489932885906
{'activation': 'tanh', 'alpha': 3.668768994992611e-08, 'hidden_layer_sizes': 88, 'random_state': 42, 'solver': 'sgd'}
第59次，测试集正确率为： 0.8389261744966443
{'activation': 'logistic', 'alpha': 2.083155801851087e-08, 'hidden_layer_sizes': 17, 'random_state': 42, 'solver': 'adam'}
第60次，测试集正确率为： 0.8389261744966443
{'activation': 'tanh', 'alpha': 7.134127680109507e-05, 'hidden_layer_sizes': 46, 'random_state': 42, 'solver': 'sgd'}
第61次，测试集正确率为： 0.8389261744966443
{'activation': 'relu', 'alpha': 0.0016314040746940018, 'hidden_layer_sizes': 95, 'random_state': 42, 'solver': 'adam'}
第62次，测试集正确率为： 0.825503355704698
{'activation': 'relu', 'alpha': 4.5027602532126234e-09, 'hidden_layer_sizes': 31, 'random_state': 42, 'solver': 'adam'}
第63次，测试集正确率为： 0.8322147651006712
{'activation': 'relu', 'alpha': 1.7617908655977186e-09

第111次，测试集正确率为： 0.8389261744966443
{'activation': 'relu', 'alpha': 1.9139481348976672e-08, 'hidden_layer_sizes': 85, 'random_state': 42, 'solver': 'sgd'}
第112次，测试集正确率为： 0.8389261744966443
{'activation': 'relu', 'alpha': 1.3904637524165077e-07, 'hidden_layer_sizes': 41, 'random_state': 42, 'solver': 'sgd'}
第113次，测试集正确率为： 0.8389261744966443
{'activation': 'relu', 'alpha': 7.649193254050383e-08, 'hidden_layer_sizes': 71, 'random_state': 42, 'solver': 'adam'}
第114次，测试集正确率为： 0.825503355704698
{'activation': 'logistic', 'alpha': 0.0002298004497341645, 'hidden_layer_sizes': 38, 'random_state': 42, 'solver': 'adam'}
第115次，测试集正确率为： 0.8456375838926175
{'activation': 'tanh', 'alpha': 1.7546917758900016e-05, 'hidden_layer_sizes': 16, 'random_state': 42, 'solver': 'sgd'}
第116次，测试集正确率为： 0.8523489932885906
{'activation': 'relu', 'alpha': 3.3611569490490636e-08, 'hidden_layer_sizes': 34, 'random_state': 42, 'solver': 'sgd'}
第117次，测试集正确率为： 0.8590604026845637
{'activation': 'relu', 'alpha': 3.43588994454

...

{'activation': 'tanh', 'alpha': 1.9896668255407446e-06, 'hidden_layer_sizes': 7, 'random_state': 42, 'solver': 'sgd'}
第9753次，测试集正确率为： 0.8859060402684564
{'activation': 'tanh', 'alpha': 1.0751510863014738e-06, 'hidden_layer_sizes': 96, 'random_state': 42, 'solver': 'sgd'}
第9754次，测试集正确率为： 0.8456375838926175
{'activation': 'tanh', 'alpha': 2.038180011774666e-06, 'hidden_layer_sizes': 29, 'random_state': 42, 'solver': 'sgd'}
第9755次，测试集正确率为： 0.8120805369127517
{'activation': 'tanh', 'alpha': 0.1472292493598639, 'hidden_layer_sizes': 2, 'random_state': 42, 'solver': 'sgd'}
第9756次，测试集正确率为： 0.6040268456375839
{'activation': 'tanh', 'alpha': 0.11295588273525313, 'hidden_layer_sizes': 32, 'random_state': 42, 'solver': 'sgd'}
第9757次，测试集正确率为： 0.8322147651006712
{'activation': 'tanh', 'alpha': 5.949520058870143e-11, 'hidden_layer_sizes': 98, 'random_state': 42, 'solver': 'sgd'}
第9758次，测试集正确率为： 0.8456375838926175
{'activation': 'tanh', 'alpha': 6.919099732591237, 'hidden_layer_sizes': 25, 'random_st

第9806次，测试集正确率为： 0.8859060402684564
{'activation': 'tanh', 'alpha': 9.731016253885693e-08, 'hidden_layer_sizes': 53, 'random_state': 42, 'solver': 'sgd'}
第9807次，测试集正确率为： 0.8456375838926175
{'activation': 'tanh', 'alpha': 3.5472178703291997, 'hidden_layer_sizes': 4, 'random_state': 42, 'solver': 'sgd'}
第9808次，测试集正确率为： 0.8456375838926175
{'activation': 'tanh', 'alpha': 0.0014165330851330872, 'hidden_layer_sizes': 7, 'random_state': 42, 'solver': 'sgd'}
第9809次，测试集正确率为： 0.8859060402684564
{'activation': 'tanh', 'alpha': 0.00039503879393240334, 'hidden_layer_sizes': 24, 'random_state': 42, 'solver': 'sgd'}
第9810次，测试集正确率为： 0.8523489932885906
{'activation': 'tanh', 'alpha': 2.5001949054364248e-05, 'hidden_layer_sizes': 10, 'random_state': 42, 'solver': 'sgd'}
第9811次，测试集正确率为： 0.825503355704698
{'activation': 'tanh', 'alpha': 1.753025372567786e-06, 'hidden_layer_sizes': 87, 'random_state': 42, 'solver': 'sgd'}
第9812次，测试集正确率为： 0.8456375838926175
{'activation': 'tanh', 'alpha': 8.141264578039704e-

第9861次，测试集正确率为： 0.8389261744966443
{'activation': 'tanh', 'alpha': 2.9498361198396374e-11, 'hidden_layer_sizes': 95, 'random_state': 42, 'solver': 'sgd'}
第9862次，测试集正确率为： 0.8523489932885906
{'activation': 'tanh', 'alpha': 2.2954080435194886e-11, 'hidden_layer_sizes': 31, 'random_state': 42, 'solver': 'sgd'}
第9863次，测试集正确率为： 0.87248322147651
{'activation': 'tanh', 'alpha': 1.0426660553420277e-11, 'hidden_layer_sizes': 92, 'random_state': 42, 'solver': 'sgd'}
第9864次，测试集正确率为： 0.8187919463087249
{'activation': 'tanh', 'alpha': 0.0804256549521535, 'hidden_layer_sizes': 33, 'random_state': 42, 'solver': 'sgd'}
第9865次，测试集正确率为： 0.8523489932885906
{'activation': 'tanh', 'alpha': 1.0938857238725771e-06, 'hidden_layer_sizes': 84, 'random_state': 42, 'solver': 'sgd'}
第9866次，测试集正确率为： 0.8322147651006712
{'activation': 'tanh', 'alpha': 0.7899027609789463, 'hidden_layer_sizes': 99, 'random_state': 42, 'solver': 'sgd'}
第9867次，测试集正确率为： 0.8389261744966443
{'activation': 'tanh', 'alpha': 1.4968814567550677e

第9915次，测试集正确率为： 0.8456375838926175
{'activation': 'tanh', 'alpha': 4.375751364345479, 'hidden_layer_sizes': 70, 'random_state': 42, 'solver': 'sgd'}
第9916次，测试集正确率为： 0.8456375838926175
{'activation': 'tanh', 'alpha': 1.7346256708516854e-12, 'hidden_layer_sizes': 7, 'random_state': 42, 'solver': 'sgd'}
第9917次，测试集正确率为： 0.8859060402684564
{'activation': 'tanh', 'alpha': 1.7461606375486702e-12, 'hidden_layer_sizes': 41, 'random_state': 42, 'solver': 'sgd'}
第9918次，测试集正确率为： 0.8523489932885906
{'activation': 'tanh', 'alpha': 0.38935624719819556, 'hidden_layer_sizes': 62, 'random_state': 42, 'solver': 'sgd'}
第9919次，测试集正确率为： 0.8322147651006712
{'activation': 'tanh', 'alpha': 1.7442927513243298e-06, 'hidden_layer_sizes': 7, 'random_state': 42, 'solver': 'sgd'}
第9920次，测试集正确率为： 0.8859060402684564
{'activation': 'tanh', 'alpha': 8.38825719529044e-07, 'hidden_layer_sizes': 52, 'random_state': 42, 'solver': 'sgd'}
第9921次，测试集正确率为： 0.8389261744966443
{'activation': 'tanh', 'alpha': 1.9370241262270394e-0

{'activation': 'tanh', 'alpha': 0.0005940872535958719, 'hidden_layer_sizes': 7, 'random_state': 42, 'solver': 'sgd'}
第9970次，测试集正确率为： 0.8859060402684564
{'activation': 'tanh', 'alpha': 0.5266983007923335, 'hidden_layer_sizes': 7, 'random_state': 42, 'solver': 'sgd'}
第9971次，测试集正确率为： 0.8859060402684564
{'activation': 'tanh', 'alpha': 1.420013127987946, 'hidden_layer_sizes': 9, 'random_state': 42, 'solver': 'sgd'}
第9972次，测试集正确率为： 0.825503355704698
{'activation': 'tanh', 'alpha': 0.3701942823113732, 'hidden_layer_sizes': 7, 'random_state': 42, 'solver': 'sgd'}
第9973次，测试集正确率为： 0.8859060402684564
{'activation': 'tanh', 'alpha': 1.3809756085672556e-08, 'hidden_layer_sizes': 78, 'random_state': 42, 'solver': 'sgd'}
第9974次，测试集正确率为： 0.8389261744966443
{'activation': 'tanh', 'alpha': 9.574862359035484e-12, 'hidden_layer_sizes': 77, 'random_state': 42, 'solver': 'sgd'}
第9975次，测试集正确率为： 0.8456375838926175
{'activation': 'tanh', 'alpha': 1.0485147140894576e-11, 'hidden_layer_sizes': 7, 'random_state':